In [1]:
import napari
import numpy as np
import skimage.filters as filters
import skimage.measure as measure
import skimage.feature as feature
import skimage.segmentation as segmentation
import scipy.ndimage as ndi

In [2]:
viewer = napari.Viewer()

In [3]:
img_dims = 256

In [4]:
img = np.zeros([img_dims, img_dims, img_dims])

In [5]:
pts = np.random.randint(0, img_dims, [1000, 3])

In [42]:
img[pts[:,0], pts[:,1], pts[:,2]] = 128

In [43]:
blurred = filters.gaussian(img, sigma=5)

In [44]:
noisy = blurred + np.random.normal(0, np.max(blurred)/100.0, img.shape)

In [56]:
viewer.add_image(noisy)

<Image layer 'noisy' at 0x7fc02840c190>

In [51]:
thresh = np.percentile(noisy, 90)

In [66]:
noisy_blurred = filters.gaussian(noisy, sigma=2)

In [67]:
mask = noisy_blurred > thresh

In [68]:
edt = ndi.distance_transform_edt(mask)

In [70]:
viewer.add_image(edt)

<Image layer 'edt' at 0x7fc028494eb0>

In [82]:
smoothed_edt = filters.gaussian(edt, sigma=0.1)

In [83]:
viewer.add_image(smoothed_edt)

<Image layer 'smoothed_edt [2]' at 0x7fbbb6f76520>

In [84]:
maxes = feature.peak_local_max(smoothed_edt, min_distance=3)

In [85]:
viewer.add_points(maxes, size=5, n_dimensional=True)

<Points layer 'maxes [3]' at 0x7fc02c178250>

In [89]:
peak_image = np.zeros(img.shape)
peak_image[maxes[:,0], maxes[:,1], maxes[:,2]] = 1
peak_image = measure.label(peak_image)

In [90]:
viewer.add_labels(peak_image)

<Labels layer 'peak_image' at 0x7fbbdacf1760>

In [91]:
watershedded = segmentation.watershed(-smoothed_edt,peak_image, mask=mask)

In [92]:
viewer.add_labels(watershedded)

<Labels layer 'watershedded' at 0x7fbbdaf62b80>